<a href="https://colab.research.google.com/github/AmandinChyba/Project1-Fields-2022/blob/main/Deep_Learning/stupid_interpolation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [453]:
import jax.numpy as jnp
import numpy as np
import jax
import jax.lax as lax

In [454]:
def getData():
  data = np.load('drive/MyDrive/wob_piecwise_data.npz')
  return data['x'], data['y']

In [455]:
def test_train_split(x, y, train_percentage=0.9):
  x = jnp.array(x)
  y = jnp.array(y)

  sample_size, dimensions = x.shape

  split_index = int(jnp.ceil(sample_size * train_percentage))

  x_train = x[:split_index]
  y_train = y[:split_index]
  x_test = x[split_index:]
  y_test = y[split_index:]

  return x_train, y_train, x_test, y_test

In [456]:
@jax.jit
def T0(x):
  return jnp.ones(len(x))

@jax.jit
def T1(x):
  return x

@jax.jit
def T2(x):
  return 2 * jnp.power(x, 2) - 1

@jax.jit
def T3(x):
  return 4 * jnp.power(x, 3) - 3 * x

In [457]:
@jax.jit
def basis(x):
  # this array creation is required because its an array of different functions at each element
  # so don't @ me
  Tx0 = jnp.array([[T0(x[0]), T1(x[0]), T2(x[0]), T3(x[0])]])
  Tx1 = jnp.array([[T0(x[1]), T1(x[1]), T2(x[1]), T3(x[1])]])
  
  big = Tx0 * Tx1.transpose(1, 0, 2)
  flat = jnp.reshape(big, (-1, big.shape[2]))
  return flat

In [458]:
def gradientDescentStep(i, c):
  f = basis(x_train.T)
  u = np.sum(c * f, axis=0, keepdims=True)
  
  c = c - lr * (u - y_train.T) * f
  c = jnp.mean(c, 1, keepdims=True)
  return c

In [459]:
%%time
c = jnp.zeros((jnp.power(4,2), 1))
lr = 0.1
epochs = 400

x, y = getData()
x_train, y_train, x_test, y_test = test_train_split(x,y)

c = lax.fori_loop(0, epochs, gradientDescentStep, c)

CPU times: user 377 ms, sys: 5.86 ms, total: 383 ms
Wall time: 390 ms


In [460]:
sample_index = 246

preds = jnp.sum(c * basis(x_test.T), axis=0)

print(y_test[sample_index])
print(preds[sample_index])

[0.97784996]
1.0458006
